In [ ]:
# Fix data.yaml paths for Kaggle and train
import os, yaml
from pathlib import Path
import torch
from ultralytics import YOLO

# Show full logs
os.environ['TQDM_DISABLE'] = '0'
os.environ.setdefault('ULTRALYTICS_HUB', '0')
os.environ.setdefault('WANDB_DISABLED', 'true')

# Kaggle inputs
yaml_src = Path('/kaggle/input/normal-data/data_merged.yaml')          # your uploaded YAML
base = Path('/kaggle/input/normal-data/merged_dataset').resolve()              # dataset root folder
ckpt = Path('/kaggle/input/test200epochmodel/pytorch/default/1/best_200.pt')      # your weights

assert base.exists(), f"Dataset root not found: {base}"
assert ckpt.exists(), f"Checkpoint not found: {ckpt}"

# Load YAML and normalize to Kaggle paths
with open(yaml_src, 'r') as f:
    data = yaml.safe_load(f) or {}

# Force dataset root and relative splits (Ultralytics resolves train/val under 'path')
data['path'] = str(base)
data['train'] = data.get('train') or 'images/train'
data['val']   = data.get('val')   or 'images/val'
# Optional: keep test if it exists; else drop to avoid missing-path warnings
if not data.get('test'):
    data.pop('test', None)

# Ensure pose keys exist (fallbacks if missing)
data.setdefault('names', ['plate'])
data.setdefault('kpt_shape', [4, 3])
data.setdefault('skeleton', [[0,1],[1,2],[2,3],[3,0]])
data.setdefault('flip_idx', [0,1,2,3])

# Write a working YAML in /kaggle/working
yaml_fixed = Path('/kaggle/working/data.yaml')
with open(yaml_fixed, 'w') as f:
    yaml.safe_dump(data, f, sort_keys=False)
print('Using data.yaml ->', yaml_fixed)
print(yaml.safe_dump(data, sort_keys=False))

# Sanity check folders
for p in [base/'images'/'train', base/'images'/'val', base/'labels'/'train', base/'labels'/'val']:
    print('Exists:', p, p.exists())

# Build model from local checkpoint
print(f'Training from: {ckpt}')
model = YOLO(str(ckpt))

# Train
train_args = dict(
    data=str(yaml_fixed),
    imgsz=768,
    epochs=200,
    patience=10,
    batch=32,
    device=0 if torch.cuda.is_available() else 'cpu',
    workers=4,             # safer on Kaggle
    project='runs',
    name='pose_plate',
    exist_ok=True,
    pretrained=False,      # no online downloads
    cache='ram',
    plots=True,
    save_period=0,
    seed=42,
    amp=False,
    verbose=True,
)

results = model.train(**train_args)
print('Training done. Best:', results)

In [ ]:
# Chạy model YOLOv8 lên ảnh image_sample.png và hiển thị kết quả với keypoint, bbox
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# Đường dẫn model và ảnh (đổi lại nếu cần)
model_path = 'best_200.pt'  # hoặc best_50.pt, best_yolo.pt tuỳ bạn
img_path = 'image_sample2.png'

assert Path(model_path).exists(), f'Không tìm thấy model: {model_path}'
assert Path(img_path).exists(), f'Không tìm thấy ảnh: {img_path}'

model = YOLO(model_path)
img = cv2.imread(img_path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

results = model(img, conf=0.25, iou=0.6, imgsz=640, max_det=5)
res = results[0]

def draw_pose(img, res, conf=0.25):
    img = img.copy()
    if hasattr(res, 'boxes') and res.boxes is not None:
        for idx, b in enumerate(res.boxes):
            c = float(b.conf[0]) if b.conf is not None else 0.0
            if c < conf: continue
            # Sửa lỗi: chuyển tensor sang numpy trước khi astype(int)
            xyxy = b.xyxy[0].cpu().numpy().astype(int) if hasattr(b.xyxy[0], 'cpu') else b.xyxy[0].astype(int)
            x1, y1, x2, y2 = xyxy
            cv2.rectangle(img, (x1, y1), (x2, y2), (0,255,0), 2)
            cv2.putText(img, f'plate {c:.2f}', (x1, max(0, y1-5)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
    if hasattr(res, 'keypoints') and res.keypoints is not None and hasattr(res.keypoints, 'xy'):
        kps_xy = res.keypoints.xy
        for pts in kps_xy:
            if pts.shape[0] >= 4:
                pts_np = pts.cpu().numpy() if hasattr(pts, 'cpu') else np.array(pts)
                for i, (px, py) in enumerate(pts_np[:4]):
                    cv2.circle(img, (int(px), int(py)), 5, (255,255,0), -1)
                    cv2.putText(img, str(i), (int(px)+5, int(py)-5), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
                cv2.polylines(img, [pts_np[:4].astype(int).reshape(-1,1,2)], True, (0,255,255), 2)
    return img

img_vis = draw_pose(img_rgb, res, conf=0.25)
plt.figure(figsize=(10,8))
plt.imshow(img_vis)
plt.axis('off')
plt.title('YOLOv8-pose result on image_sample.png')
plt.show()